In [ ]:
%reload_ext autoreload 
%autoreload 2  #파이썬 코드를 실행하기 전에 항상 모든 모듈을 Reload하라는 의미
%matplotlib inline #브라우저내부에 그림 바로 보여지게 하는 코드

In [21]:
from matplotlib import pyplot as plt #matplotlib; 패키지 한글 깨짐 처리 시작
from matplotlib import rcParams
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
from pathlib import Path
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
import seaborn as sns
import warnings

In [5]:
rcParams['figure.figsize'] = (16, 8) # 그래프에서 폰트 깨지는 문제에 대한 대처(그림(figure)의 크기. (가로,세로) 인치 단위)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)#컬럼최대수 지정
pd.set_option("display.precision", 4)#소수점넷째짜리 밑까지 표시
warnings.simplefilter('ignore')

#학습데이터 로드하기

In [13]:
data_dir = Path('../data/dacon-author-classification')
feature_dir = Path('../build/feature')
val_dir = Path('../build/val')
tst_dir = Path('../build/tst')
sub_dir = Path('../build/sub')

trn_file =  'c:/train.csv'
tst_file = 'c:/test_x.csv'
sample_file = 'c:/sample_submission.csv'

target_col = 'author'
n_fold = 5
n_class = 5
seed = 42

In [14]:
algo_name = 'lr'
feature_name = 'tfidf'
model_name = f'{algo_name}_{feature_name}'

feature_file = f'{feature_name}.csv'
p_val_file =   f'{model_name}.val.csv'
p_tst_file =  f'{model_name}.tst.csv'
sub_file =  f'{model_name}.csv'

In [15]:
import pandas as pd

trn = pd.read_csv('c:/train.csv')
print(trn.shape)
trn.head()

(54879, 3)


,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3


In [16]:
tst = pd.read_csv('c:/test_x.csv')
print(tst.shape)
tst.head()

(19617, 2)


,index,text
0,0,“Not at all. I think she is one of the most ch...
1,1,"""No,"" replied he, with sudden consciousness, ""..."
2,2,As the lady had stated her intention of scream...
3,3,“And then suddenly in the silence I heard a so...
4,4,His conviction remained unchanged. So far as I...


In [23]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.stem.snowball import SnowballStemmer


In [18]:
s = trn.text[4]
print(s)

“Have mercy, gentlemen!” odin flung up his hands. “Don’t write that, anyway; have some shame. Here I’ve torn my heart asunder before you, and you seize the opportunity and are fingering the wounds in both halves.... Oh, my God!”


In [25]:
tokens = word_tokenize(s)
print(tokens)


['“', 'Have', 'mercy', ',', 'gentlemen', '!', '”', 'odin', 'flung', 'up', 'his', 'hands', '.', '“', 'Don', '’', 't', 'write', 'that', ',', 'anyway', ';', 'have', 'some', 'shame', '.', 'Here', 'I', '’', 've', 'torn', 'my', 'heart', 'asunder', 'before', 'you', ',', 'and', 'you', 'seize', 'the', 'opportunity', 'and', 'are', 'fingering', 'the', 'wounds', 'in', 'both', 'halves', '....', 'Oh', ',', 'my', 'God', '!', '”']


In [28]:
lemmatizer = WordNetLemmatizer()
[lemmatizer.lemmatize(t) for t in tokens]

['“',
 'Have',
 'mercy',
 ',',
 'gentleman',
 '!',
 '”',
 'odin',
 'flung',
 'up',
 'his',
 'hand',
 '.',
 '“',
 'Don',
 '’',
 't',
 'write',
 'that',
 ',',
 'anyway',
 ';',
 'have',
 'some',
 'shame',
 '.',
 'Here',
 'I',
 '’',
 've',
 'torn',
 'my',
 'heart',
 'asunder',
 'before',
 'you',
 ',',
 'and',
 'you',
 'seize',
 'the',
 'opportunity',
 'and',
 'are',
 'fingering',
 'the',
 'wound',
 'in',
 'both',
 'half',
 '....',
 'Oh',
 ',',
 'my',
 'God',
 '!',
 '”']

In [29]:
stemmer = SnowballStemmer("english")
[stemmer.stem(t) for t in tokens]


['“',
 'have',
 'merci',
 ',',
 'gentlemen',
 '!',
 '”',
 'odin',
 'flung',
 'up',
 'his',
 'hand',
 '.',
 '“',
 'don',
 '’',
 't',
 'write',
 'that',
 ',',
 'anyway',
 ';',
 'have',
 'some',
 'shame',
 '.',
 'here',
 'i',
 '’',
 've',
 'torn',
 'my',
 'heart',
 'asund',
 'befor',
 'you',
 ',',
 'and',
 'you',
 'seiz',
 'the',
 'opportun',
 'and',
 'are',
 'finger',
 'the',
 'wound',
 'in',
 'both',
 'halv',
 '....',
 'oh',
 ',',
 'my',
 'god',
 '!',
 '”']

#Bag-of-Words 피처 생성

In [35]:
vec = CountVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 2), min_df=100)
X_cnt = vec.fit_transform(trn['text'])
print(X_cnt.shape)
#카운트 벡터라이저진행

(54879, 2685)


In [36]:
X_cnt[0, :50].todense()
#행렬을 SUM하는 것

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [37]:
vec = TfidfVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 3), min_df=50)
X = vec.fit_transform(trn['text'])
X_tst = vec.transform(tst['text'])
print(X.shape, X_tst.shape)

#TFIDF 벡터라이저 진행

(54879, 5897) (19617, 5897)


In [38]:
X[0, :50].todense()

matrix([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.]])

#로지스틱 회귀모델 학습

In [39]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

#교차 검증 반복자(iterators) 로서 KFold, StratifiedKFold를 많이 사용합니다.
#비슷하면서도 다른점 있습니다.
#교차 유효성 검사 전략에 따라 데이터 집합을 생성하는 데 사용할 수 있는 인덱스를 생성하는 도구 입니다.

In [40]:
y = trn.author.values
y.shape

(54879,)

In [41]:
p = np.zeros((X.shape[0], n_class))
#zero()함수는 0으로 초기화된 shape 차원의 ndarray 배열 객체를 반환
p_tst = np.zeros((X_tst.shape[0], n_class))
#enumerate는 "열거하다"라는 뜻이다. 이 함수는 순서가 있는 자료형(리스트, 튜플, 문자열)을 입력으로 받아 인덱스 값을 포함하는 enumerate 객체를 돌려준다.
for i_cv, (i_trn, i_val) in enumerate(cv.split(X, y), 1):
    clf = LogisticRegression()
    clf.fit(X[i_trn], y[i_trn])
    p[i_val, :] = clf.predict_proba(X[i_val])
    p_tst += clf.predict_proba(X_tst) / n_class
    #predict_proba: 분류기에서 불확실성을 추정
    # predict_proba의 출력은 각 클래스에 대한 확률

In [42]:
print(f'Accuracy (CV): {accuracy_score(y, np.argmax(p, axis=1)) * 100:8.4f}%')
print(f'Log Loss (CV): {log_loss(pd.get_dummies(y), p):8.4f}')

Accuracy (CV):  76.6140%
Log Loss (CV):   0.6800


In [43]:
np.savetxt(p_val_file, p, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, p_tst, fmt='%.6f', delimiter=',')

#제출 파일 생성

In [44]:
sub = pd.read_csv(sample_file, index_col=0)
print(sub.shape)
sub.head()

(19617, 5)


,0,1,2,3,4
index,,,,,
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0


In [45]:
sub[sub.columns] = p_tst
sub.head()

,0,1,2,3,4
index,,,,,
0,0.0631,0.5302,0.3155,0.0659,0.0253
1,0.0815,0.8202,0.0032,0.0269,0.0682
2,0.7208,0.0319,0.1174,0.0381,0.0918
3,0.0392,0.0036,0.8465,0.0058,0.1049
4,0.3044,0.2440,0.1450,0.1905,0.1161


In [ ]:
sub.to_csv(sub_file)